# GloVe Corpus

Reference: https://nlp.stanford.edu/projects/glove/

## Import Libraries

In [1]:
import pandas as pd
import numpy as np

from nltk.tokenize import word_tokenize
import re
import string

from sklearn.model_selection import train_test_split

import tensorflow as tf

## Define Functions

In [2]:
def getGloveCorpus(dims=100):
    # Set path and load corpus
    path = './Datasets/'
    filename = f'glove.6B.{dims}d.txt'
    f = open(path+filename, 'r', encoding='latin2')
    vec_txt = f.read()

    vec_data = {}
    words = vec_txt.split('\n')
    for word in words:
        vec = word.split()
        if len(vec) == dims+1:
            vec_data[vec[0]] = np.array([np.float16(x) for x in vec[1:]])
    vec = pd.DataFrame(vec_data, columns=None).transpose()
    return vec


def cleanText(textData):
    cleanText = []

    for i in textData:
        # convert to lowercase
        new = i.lower()

        # remove urls
        url = re.compile(r'https?://\S+|www\.\S+')
        new = url.sub(r'', new)

        # remove html tags
        html = re.compile(r'<.*?>')
        new = html.sub(r'', new)

        # remove punctuation
        operator = str.maketrans('', '', string.punctuation)
        new = new.translate(operator)

        cleanText.append(new)

    return cleanText

def getGloveVec(word, vec, dims=300):
    vc = np.zeros(dims)
    try:
        vc = np.array(vec.loc[word])
    except:
        vc = np.zeros(dims)
    return vc


def getDocVec(sentence, vec, dims):
    tokens = word_tokenize(sentence)
    vecs = np.zeros(dims)
    for word in tokens:
        vecs += getGloveVec(word, vec, dims)

    return vecs


def encodeClass(Y):
    label = []
    classes = Y.unique()
    for i in Y:
        label.append(np.where(classes == i)[0][0])
    return label


def getVecForm(X, Y, dims, vec, reshaping=None):
    '''
    X - Array of sentences.
    Y - Array of output class (numeric).
    dims - Vector dimensions (needed to pad correctly).
    '''
    vecList = []
    for i in X:
        vecList.append(getDocVec(i, vec, dims))
    X = np.asarray(vecList).astype(np.float16)
    Y = np.asarray(encodeClass(Y)).astype(np.float16)
    if reshaping != None:
        X = np.reshape(X, reshaping)
    return X, Y


## Set Configuration

In [3]:
PATH = './Datasets/'
DIMS = 100
MAXVECLEN = 100
DATASET = 'TARP_Project_Final_Dataset.csv'
TEXT = 'title'
CLASS = 'label'
EPOCH = 150
SPLIT = 0.25

## Read GloVe Data

In [4]:
# VEC = getGloveCorpus(dims=DIMS)
VEC = pd.read_csv('Glove.csv', index_col=0)
VEC

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
the,-0.03820,-0.24490,0.7280,-0.39970,0.0832,0.04395,-0.39140,0.33450,-0.57570,0.08746,...,0.01622,-0.017100,-0.39000,0.8740,-0.72560,-0.51070,-0.52050,-0.1459,0.8276,0.27050
",",-0.10767,0.11053,0.5980,-0.54350,0.6740,0.10660,0.03888,0.35470,0.06354,-0.09420,...,0.34960,-0.722700,0.37550,0.4440,-0.99070,0.61230,-0.35100,-0.8315,0.4530,0.08260
.,-0.33980,0.20940,0.4634,-0.64800,-0.3838,0.03802,0.17130,0.15980,0.46630,-0.01917,...,-0.06335,-0.674300,-0.06890,0.5360,-0.87800,0.31800,-0.39230,-0.2339,0.4730,-0.02881
of,-0.15300,-0.24280,0.8984,0.16990,0.5350,0.48780,-0.58840,-0.17980,-1.35800,0.42530,...,0.18710,-0.018500,-0.26760,0.7270,-0.59380,-0.34840,-0.56100,-0.5910,1.0040,0.20670
to,-0.18970,0.05002,0.1908,-0.04920,-0.0897,0.21010,-0.54930,0.09840,-0.20130,0.34250,...,-0.13130,0.058620,-0.31860,-0.6143,-0.62400,-0.41550,-0.03818,-0.3980,0.4766,-0.15980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
chanty,-0.15580,-0.04920,-0.0644,0.22360,-0.2014,-0.03897,0.12980,-0.29440,0.00359,-0.09840,...,0.09330,0.094500,-0.02347,-0.4810,0.62350,0.02432,-0.27600,0.0751,-0.5640,0.14500
kronik,-0.09440,0.14720,-0.1573,0.07196,-0.2983,0.03943,0.02187,0.00804,-0.18680,-0.31100,...,-0.30540,-0.011086,0.11850,-0.1131,0.33960,-0.22450,0.25730,0.6313,-0.2009,-0.10540
rolonda,0.36080,-0.16920,-0.3271,0.09830,-0.4297,-0.18870,0.45560,0.28540,0.30350,-0.36700,...,-0.04407,0.140000,0.30000,-0.1273,-0.14310,-0.06940,0.28150,0.2715,-0.2920,0.16110
zsombor,-0.10460,-0.50500,-0.4934,0.13510,-0.3638,-0.44750,0.18430,-0.05652,0.40480,-0.72560,...,0.15150,-0.108400,0.34060,-0.4092,-0.08124,0.09534,0.15010,0.4253,-0.5127,-0.17050


## Import Dataset

In [5]:
df = pd.read_csv(PATH+DATASET).dropna().reset_index(drop=True)
df

,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL
...,...,...,...
61139,Powell pushes diplomacy for N. Korea,WASHINGTON -- Outgoing Secretary of State Coli...,REAL
61140,Void is filled with Clement,With the supply of attractive pitching options...,REAL
61141,Martinez leaves bitter,Like Roger Clemens did almost exactly eight ye...,REAL
61142,5 of arthritis patients in Singapore take Bext...,SINGAPORE : Doctors in the United States have ...,REAL


### Remove excessively long texts

In [6]:
tooLong = []
for i in range(len(df[TEXT])):
    if len(df[TEXT][i].split()) > MAXVECLEN:
        tooLong.append(i)
for i in tooLong:
    df = df.drop(i, axis=0)
df = df.reset_index(drop=True)
df

,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL
...,...,...,...
61139,Powell pushes diplomacy for N. Korea,WASHINGTON -- Outgoing Secretary of State Coli...,REAL
61140,Void is filled with Clement,With the supply of attractive pitching options...,REAL
61141,Martinez leaves bitter,Like Roger Clemens did almost exactly eight ye...,REAL
61142,5 of arthritis patients in Singapore take Bext...,SINGAPORE : Doctors in the United States have ...,REAL


## Clean Text

In [7]:
df[TEXT] = cleanText(df[TEXT])
df

,title,text,label
0,you can smell hillary’s fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,watch the exact moment paul ryan committed pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,kerry to go to paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,bernie supporters on twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,the battle of new york why this primary matters,It's primary day in New York and front-runners...,REAL
...,...,...,...
61139,powell pushes diplomacy for n korea,WASHINGTON -- Outgoing Secretary of State Coli...,REAL
61140,void is filled with clement,With the supply of attractive pitching options...,REAL
61141,martinez leaves bitter,Like Roger Clemens did almost exactly eight ye...,REAL
61142,5 of arthritis patients in singapore take bext...,SINGAPORE : Doctors in the United States have ...,REAL


In [8]:
X_, Y_ = df[TEXT], df[CLASS]

## Simple NN

### Convert dataset into Word Vector

In [9]:
X, Y = getVecForm(
    X = X_,
    Y = Y_,
    vec = VEC,
    dims=DIMS
)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
X.shape

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y)

### Training a Simple NN

In [ ]:
def simpleNN(X):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(
        128, activation='relu', input_shape=X.shape[1:]))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    opt = tf.keras.optimizers.Adam(learning_rate=0.0005)
    model.compile(loss='binary_crossentropy',
                  optimizer=opt, metrics=['accuracy'])
    return model

In [ ]:
snn = simpleNN(X)
snn.fit(x_train, y_train, validation_split=SPLIT, epochs=EPOCH)

In [ ]:
snn.evaluate(x_test, y_test)

In [ ]:
snn.save('FakeNewsANN.h5')

## Simple LSTM (sum)

### Convert dataset into Word Vector

In [ ]:
X = np.reshape(X, (X.shape[0], 1, X.shape[1]))
X.shape

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y)

### Training a Simple LSTM

In [ ]:
def simpleLSTM(X):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.LSTM(8, input_shape=X.shape[1:]))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(32, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.2))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    opt = tf.keras.optimizers.Adam(learning_rate=0.0005)
    model.compile(loss='binary_crossentropy',
                  optimizer=opt, metrics=['accuracy'])
    return model

In [ ]:
lstms = simpleLSTM(X)
lstms.fit(x_train, y_train, validation_split=SPLIT, epochs=EPOCH)

In [ ]:
lstms.evaluate(x_test, y_test)